## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, mean_squared_error
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Look at the first 5 rows
model_data.head()

,Angaus,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.0,-0.10,1.036,50.20,0.57,0.09,2.470,9.8,0.81,0,electric,4.8
1,1,18.7,1.51,1.003,132.53,1.15,0.20,1.153,8.3,0.34,0,electric,2.0
2,0,18.3,0.37,1.001,107.44,0.57,0.49,0.847,0.4,0.00,0,spo,1.0
3,0,16.7,-3.80,1.000,166.82,1.72,0.90,0.210,0.4,0.22,1,electric,4.0
4,1,17.2,0.33,1.005,3.95,1.15,-1.20,1.980,21.9,0.96,0,electric,4.7


In [3]:
# Encode categorical variable
le = LabelEncoder()
model_data["Method"] = le.fit_transform(model_data["Method"])

# Verify
model_data.dtypes

Angaus          int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method          int64
LocSed        float64
dtype: object

In [4]:
# Define target and feature variables
X = model_data.drop(columns = "Angaus")
y = model_data["Angaus"]

In [5]:
# Split data into training and validation 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [6]:
# Initialize model with hyperparameters
xg = XGBClassifier(n_estimators = 1000,
                   learning_rate = 0.1,
                   eval_metric = "logloss",
                   early_stopping_rounds = 50,
                   random_state = 808)

# Fit training data with evaluation parameter
xg.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

# Print best number of trees
best_number_trees = xg.best_iteration
print(f"Best Number of Trees: {best_number_trees}")

Best Number of Trees: 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [7]:
# Define paramter distribution
param_distributions = {
    "learning_rate": uniform(0.01, 0.3)
}

# Use RandomizedSearchCV
rs = RandomizedSearchCV(xg, 
                        n_iter = 20, 
                        param_distributions = param_distributions,
                        cv = 5,
                        scoring = "neg_log_loss",
                        n_jobs = -1,
                        random_state = 808)
# Carry out the search
rs.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

# Print best learning rate
best_learning_rate = rs.best_params_["learning_rate"]
print(f"Best Learning Rate: {best_learning_rate:.4f}")

Best Learning Rate: 0.2204


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [8]:
# Initialize model with best number of trees and learning rate
xg = XGBClassifier(n_estimators = best_number_trees,
                   learning_rate = best_learning_rate,
                   eval_metric = "logloss",
                   early_stopping_rounds = 50,
                   random_state = 808)

# Define a parameter dictionary
param_dic = {
    "max_depth": randint(3, 11),
    "min_child_weight": randint(1, 11),
    "gamma": uniform(0.05, 0.10)
}

# Use RandomizedSearchCV
rs = RandomizedSearchCV(xg, 
                        n_iter = 20, 
                        param_distributions = param_dic,
                        cv = 5,
                        scoring = "neg_log_loss",
                        n_jobs = -1,
                        random_state = 808)

# Carry out the search
rs.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

# Print best parameters
best_gamma = rs.best_params_["gamma"]
best_depth = rs.best_params_["max_depth"]
best_weight = rs.best_params_["min_child_weight"]
print(f"Best Gamma: {best_gamma:.4f}")
print(f"Best Max Depth: {best_depth}")
print(f"Best Min Child Weight: {best_weight}")

Best Gamma: 0.0650
Best Max Depth: 5
Best Min Child Weight: 8


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [9]:
# Initialize model with best parameters
xg = XGBClassifier(n_estimators = best_number_trees,
                   learning_rate = best_learning_rate,
                   max_depth = best_depth,
                   min_child_weight = best_weight,
                   gamma = best_gamma,
                   eval_metric = "logloss",
                   early_stopping_rounds = 50,
                   random_state = 808)

# Define a parameter dictionary
param_dic = {
    "subsample": uniform(0.05, 0.10),
    "colsample_bytree": uniform(0.05, 0.10)
}

# Use RandomizedSearchCV
rs = RandomizedSearchCV(xg, 
                        n_iter = 20, 
                        param_distributions = param_dic,
                        cv = 5,
                        scoring = "neg_log_loss",
                        n_jobs = -1,
                        random_state = 808)

# Carry out the search
rs.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

# Print best parameters
best_colsample = rs.best_params_["colsample_bytree"]
best_subsample = rs.best_params_["subsample"]
print(f"Best Colsample By Tree: {best_colsample:.4f}")
print(f"Best Subsample: {best_subsample:.4f}")

Best Colsample By Tree: 0.1475
Best Subsample: 0.1381


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [10]:
# Look at the first 5 rows
eval_data.head()

,Angaus_obs,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.6,1.01,1.017,5.23,0.29,-1.40,1.980,10.0,1.00,0,electric,4.9
1,1,16.8,-0.51,1.002,2.24,0.00,0.27,0.460,0.7,0.00,0,electric,2.3
2,0,16.3,0.76,1.023,162.28,5.14,-0.60,0.806,21.4,0.66,0,electric,4.3
3,0,15.6,1.56,1.003,4.05,0.57,1.14,3.300,0.9,0.75,0,net,1.0
4,0,14.6,-0.20,1.023,127.03,1.72,-1.90,1.940,28.9,0.97,0,electric,NaN


In [11]:
# Encode categorical variable
le = LabelEncoder()
eval_data["Method"] = le.fit_transform(eval_data["Method"])

# Verify
eval_data.dtypes

Angaus_obs      int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method          int64
LocSed        float64
dtype: object

In [12]:
# Define target and feature variables
X_eval = eval_data.drop(columns = "Angaus_obs")
y_eval = eval_data["Angaus_obs"]

In [13]:
# Initialize model with best parameters
xg = XGBClassifier(n_estimators = best_number_trees,
                   learning_rate = best_learning_rate,
                   max_depth = best_depth,
                   min_child_weight = best_weight,
                   gamma = best_gamma,
                   colsample_bytree = best_colsample,
                   subsample = best_subsample,
                   early_stopping_rounds = 50,
                   eval_metric = "logloss",
                   random_state = 808)

# Carry out the search
xg.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

# Predict on the evaluation data
y_pred_prob = xg.predict_proba(X_eval)[:, 1]

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [14]:
# Compute and print AUC score 
auc = roc_auc_score(y_eval, y_pred_prob)
print(f"AUC: {auc:.4f}")

# Compute feature importances
feature_importances = xg.feature_importances_

# Create a df with name of feature
importances_df = pd.DataFrame({
    "Feature": X_eval.columns,
    "Importance": feature_importances
})

# Sort in descending of importance
importances_df = importances_df.sort_values(by = "Importance", ascending = False)

# Print importances
importances_df

AUC: 0.7990


,Feature,Importance
11,LocSed,0.302320
0,SegSumT,0.256970
8,USNative,0.151296
7,USSlope,0.094856
2,SegLowFlow,0.081389
6,USRainDays,0.045797
5,USAvgT,0.034035
1,SegTSeas,0.033336
3,DSDist,0.000000
4,DSMaxSlope,0.000000


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

Compared to Elith et al., my model's performance is not as strong. The AUC score of my model is 0.7990, which is lower than the CV AUC of 0.869 reported in their study. Additionally, the ranking of feature importance differs: while Elith et al. found SegSumT to be the most important feature, LocSed is ranked highest in my model. However another way we can compare models is by its complexity is by calculatiing the mean squared error (MSE). In this case, Elith et al.'s model still wins in that regard by having a lower MSE of 0.015.

In [15]:
# Evaluate model performance (MSE)
mse = mean_squared_error(y_eval, xg.predict_proba(X_eval)[:, 1])
print(f"MSE: {mse:.4f}")

MSE: 0.1414
